# Cox proportional hazards models for T-ALL paper

Select kernel: Single Cell R

* Fig4D & SuppTable 11
* Fig4E & SuppTable 13
* SuppFig6A & SuppTable12

### Set up

In [1]:
### suppressMessages(source("/nfs/users/nfs_h/hw12/T_ALL/notebooks/T_helper_functions.R"))
requiredPackages <- c("tidyverse", "ggpubr","patchwork","ggrepel",
                     "circlize","ComplexHeatmap", "GSEABase", "edgeR","limma",
                     "survminer","survival", "plyr","ggalt","ggbeeswarm") #"Manu",

for (pkg in requiredPackages){
  suppressWarnings(suppressMessages(library(pkg, character.only = T)))
}

source("/nfs/users/nfs_h/hw12/processing_scripts/survival_analysis/GDC_signature_survival_functions.R")
source("/nfs/users/nfs_h/hw12/processing_scripts/survival_analysis/GDC_signature_survival_functions_final.R")

fig <- function(width, heigth){
 ## From - https://stackoverflow.com/questions/45473128/r-changing-ggplot-plot-size-in-jupyter
 options(repr.plot.width = width, repr.plot.height = heigth)
 }

### --- Load modules
Z_modules <- getGmt(paste0("/lustre/scratch127/recovered/scratch126/lustre/scratch126/casm/team274sb/hw12/T_ALL/modules/", 
                     "ZBTB16_modules_refined.gmt"))
### --- Load data
#load(file="/lustre/scratch126/casm/team274sb/project_folders/T_ALL_paper/data/UPenn_dge.Rdata")
load( file="/nfs/team274/bl10/TALL_bulkRNA_UPenn/UPenn_dge.Rdata")


### --- Add survival info
#load_dir <- "/lustre/scratch126/casm/team274sb/bl10/T-ALL/Data/Bulk_RNA/UPennFINAL/"
load_dir = "/nfs/team274/bl10/TALL_bulkRNA_UPenn/"
bulk_metadat_1 = read.table(paste0(load_dir,"ST3_Genomic_Data.csv"),sep=",", header=TRUE)
bulk_metadat_2 = read.table(paste0(load_dir,"ST1_Clinical_Data.csv"),sep=",", header=TRUE)

bulk_metadat_2$DueToDisease <- ifelse(bulk_metadat_2$Cause.of.Death %in% c("; Due to this disease","Due to this disease;",
                                                                           "Due to this disease"," Due to this disease"),
                                      "Due to this disease","Not")



for (col_str in c('OS','OS.status','EFS','EFS.status','DFS','DFS.status','DueToDisease')){
    UPenn_dge$samples[[col_str]] <- as.vector(setNames(bulk_metadat_2[[col_str]],
                                    bulk_metadat_2$USI)[UPenn_dge$samples$sample_id])
}

save_dir <- "/lustre/scratch125/cellgen/behjati/hw12/T_ALL/survival_output/"

In [2]:
plotGDCSurvival_hjw <- function(survdf_list,n_row=1, str="ModuleScore", y_str=""){

  plotdf = ldply(survdf_list, .id = 'Project')
  
  #order by pvals
  pvaldf = base::unique(plotdf[, c('Project', 'p')])
  plotdf$Project = factor(plotdf$Project, pvaldf$Project[order(pvaldf$p)])
  
  #add counts
  plotdf = ddply(plotdf, 'Group', function (x) {
    x$Group = paste0(x$Group, ' ~ ', str,' (n=', nrow(x), ')')
    return(x)
  })
  
  #pvalue annotations
  annotdf = base::unique(plotdf[, c('Project', 'p_txt')])
  annotdf$Group = plotdf$Group[1]
  
  p1 = ggplot(plotdf, aes(Time, surv, colour = Group, fill = Group)) +
    geom_ribbon(
      aes(ymin = lower, ymax = upper),
      colour = NA,
      alpha = 0.25,
      stat = 'stepribbon',
      show.legend = FALSE
    ) +
    geom_step(show.legend = FALSE) +
    geom_point(data = plotdf[plotdf$Event != 0, ],
               shape = '+',
               size = 5) +
    geom_text(
      aes(x = -Inf, y = -Inf, label = p_txt),
      hjust = -0.1,
      vjust = -1,
      colour = 'black',
      data = annotdf,
      show.legend = FALSE
    ) +
    facet_wrap(~ Project, scales = 'free_x',nrow=n_row) +
    xlab('Time') +
    scale_y_continuous(limits = c(0, 1),
                       name = paste0('Survival probability',y_str),
                       breaks = c(0, 0.5, 1)) +
    scale_colour_manual(
     # palette ='Set2',#'Set2',
      values=c( "#8E1F43","#237186", "#DCC949"),
      aesthetics = c('colour', 'fill'),
      # label = function(breaks) {parse(text = breaks)},
      guide = guide_legend(
        override.aes = list(shape = '+', linetype = 1),
        ncol = 2,
        title = ''
      )
    ) +
    guides(fill = guide_none()) + theme_bw() +
    theme(legend.position = 'bottom', axis.text=element_text(size=12),axis.title=element_text(size=16),
          legend.text=element_text(size=14), strip.background = element_blank(),strip.text.x=element_text(size=18))
  return(p1)
}

runSurvivalAnalysis <- function(cdata_x,cluster, event_str){
    #cdata_x = UPenn_dge$samples
    
    ## -- Check values
    if (length(cdata_x[which((cdata_x[, event_str] < 0) == TRUE), 
                   "short_name"]) > 0 ){
        print("Some patients have negative values for event")
    }
    
    ## -- Create into survival data structure
    cdata_x$s = grepl("1", cdata_x$OS.status, ignore.case = TRUE)
    cdata_x$cluster = as.factor(cluster)
    cdata_x$event <- as.vector(cdata_x[[event_str]])
    cdata_x = cdata_x[, c("event", "s", "cluster")]
    
    ## -- Set formula
    f = as.formula("Surv(event, s) ~ cluster")
    #f = as.formula("Surv(event) ~ cluster")
    sfit = do.call(survival::survfit, list(formula = f, data = cdata_x))
    
    ## -- Build DF
    strata = sfit$strata
    names(strata) = gsub('cluster=', '', names(strata))

    strata = rep(names(strata), times = strata)
    plotdf = data.frame(
        'Time' = sfit$time,
        'surv' = sfit$surv,
        'Event' = sfit$n.censor,
        'lower' = sfit$lower,
        'upper' = sfit$upper,
        'Group' = strata,
        'p' = surv_pvalue(sfit, data = cdata_x)$pval,
        'p_txt' = surv_pvalue(sfit, data = cdata_x)$pval.txt
    )
    #plotdf = plotdf[!is.na(plotdf$lower), ]
    #stopifnot(all(table(plotdf$Group) > 1))
    
    survdfs = list(T_ALL=plotdf)
    survdfs = survdfs[!is.na(survdfs)]
    
    ## -- Return
    return(survdfs)
    
}

runSurvivalAnalysis_EFS <- function(cdata_x, cluster, event_str, status_str){
   # cdata_x = UPenn_dge$samples
    
    ## -- Check values
    if (length(cdata_x[which((cdata_x[, event_str] < 0) == TRUE), 
                   "short_name"]) > 0 ){
        print("Some patients have negative values for event")
    }
    
    ## -- Create into survival data structure
    cdata_x$s = grepl("1", cdata_x[[status_str]], ignore.case = TRUE)
    cdata_x$cluster = as.factor(cluster)
    cdata_x$event <- as.vector(cdata_x[[event_str]])
    cdata_x = cdata_x[, c("event", "s", "cluster")]
    
    ## -- Set formula
    f = as.formula("Surv(event, s) ~ cluster")
    #f = as.formula("Surv(event) ~ cluster")
    sfit = do.call(survival::survfit, list(formula = f, data = cdata_x))
    
    ## -- Build DF
    strata = sfit$strata
    names(strata) = gsub('cluster=', '', names(strata))

    strata = rep(names(strata), times = strata)
    plotdf = data.frame(
        'Time' = sfit$time,
        'surv' = sfit$surv,
        'Event' = sfit$n.censor,
        'lower' = sfit$lower,
        'upper' = sfit$upper,
        'Group' = strata,
        'p' = surv_pvalue(sfit, data = cdata_x)$pval,
        'p_txt' = surv_pvalue(sfit, data = cdata_x)$pval.txt
    )
    #plotdf = plotdf[!is.na(plotdf$lower), ]
    #stopifnot(all(table(plotdf$Group) > 1))
    
    survdfs = list(T_ALL=plotdf)
    survdfs = survdfs[!is.na(survdfs)]
    
    ## -- Return
    return(survdfs)
    
}


<br>

<br>

# Fig 4D & SuppTable11

### Split data

In [3]:
cutquantile = 0.3333

## -- By ZBTB16

scvec_Z = UPenn_dge$logCPM["ZBTB16",]
scstrata_Z = rep(NA, length(scvec_Z))
scstrata_Z[scvec_Z < quantile(scvec_Z, cutquantile)] = 'Low'
scstrata_Z[scvec_Z > quantile(scvec_Z, 1 - cutquantile)] = 'High'

## -- Rename
scstrata_Z = gsub('High', ' ^ Hi', scstrata_Z)
scstrata_Z = gsub('Low', ' ^ Lo', scstrata_Z)
names(scstrata_Z) <- names(scvec_Z)

UPenn_dge$samples$survival_group <- as.vector(scstrata_Z[UPenn_dge$samples$sample_id])
UPenn_dge$samples$ZBTB16_exprs <-  as.vector(UPenn_dge$logCPM["ZBTB16",])


## -- By ETP  // exclude Near-ETP & Unknown

scstrata_ETP = rep(NA, nrow(UPenn_dge$samples))
scstrata_ETP[UPenn_dge$samples$ETP_status=="ETP"] = 'ETP'
scstrata_ETP[UPenn_dge$samples$ETP_status=="Non-ETP"] = 'Non-ETP'
names(scstrata_ETP) <- rownames(UPenn_dge$samples)

## -- By ETP  // exclude unknown but include Near-ETP!
scstrata_ETP_near = rep(NA, nrow(UPenn_dge$samples))
scstrata_ETP_near[UPenn_dge$samples$ETP_status=="ETP"] = 'ETP'
scstrata_ETP_near[UPenn_dge$samples$ETP_status %in% c("Non-ETP", "Near-ETP")] = 'Non-ETP'
names(scstrata_ETP_near) <- rownames(UPenn_dge$samples)




### Run cox

In [4]:
testCOX <- function(ZBTB16_vec_x, include_near=FALSE, status_str="OS.status", event_str = "OS"){
    ### Alternative:
    # - status_str = "EFS.status"
    # - event_str = "EFS"
    
    ## -- Get ETP vector
    if (include_near==FALSE){
        ETP_vec_x = rep(NA, nrow(UPenn_dge$samples))
        ETP_vec_x[UPenn_dge$samples$ETP_status=="ETP"] = 'ETP'
        ETP_vec_x[UPenn_dge$samples$ETP_status=="Non-ETP"] = 'Non-ETP'
        names(ETP_vec_x) <- rownames(UPenn_dge$samples)
    } else {
        ETP_vec_x = rep(NA, nrow(UPenn_dge$samples))
        ETP_vec_x[UPenn_dge$samples$ETP_status=="ETP"] = 'ETP'
        ETP_vec_x[UPenn_dge$samples$ETP_status %in% c("Non-ETP", "Near-ETP")] = 'Non-ETP'
        names(ETP_vec_x) <- rownames(UPenn_dge$samples)
    }

    
    ## -- Prep
    cdata_x = UPenn_dge$samples[rownames(UPenn_dge$samples) %in% names(ETP_vec_x[!(is.na(ETP_vec_x))]),]



    ## -- Check values
    if (length(cdata_x[which((cdata_x[, event_str] < 0) == TRUE), 
                   "short_name"]) > 0 ){
            print("Some patients have negative values for event")
    }
    
    # -- Create into survival data structure
    cdata_x$s = grepl("1", cdata_x[[status_str]], ignore.case = TRUE)
    cdata_x$ETP = factor(as.vector(ETP_vec_x[rownames(cdata_x)]), levels=unique(as.vector(ETP_vec_x[rownames(cdata_x)])))

    if (is.character(ZBTB16_vec_x[[1]])){
        cdata_x$Z_expres <- factor(as.vector(ZBTB16_vec_x[rownames(cdata_x)]))
        cdata_x$Z_expres <- relevel(cdata_x$Z_expres, " ^ Lo")
    } else {
        cdata_x$Z_expres = as.vector(ZBTB16_vec_x[rownames(cdata_x)])
    }

    cdata_x$event <- as.vector(cdata_x[[event_str]])
    cdata_x = cdata_x[, c("event", "s", "ETP","Z_expres")]
    cdata_x = cdata_x[order(cdata_x$ETP),]

    #if (class(cdata_x$Z_expres)=="numeric"){
    #    Z_str = "Z_expres"
    #} else {
    #    cdata_x$Z_expres <- as.factor(cdata_x$Z_expres)
    #    cdata_x = cdata_x[rev(order(cdata_x$Z_expres)),]
    #    Z_str = paste0("Z_expres",unique(cdata_x$Z_expres[!(is.na(cdata_x$Z_expres))])[2])    
    #}
    
    ## -- Set formula
    f = as.formula("Surv(event, s) ~ ETP + Z_expres")
    
    sfit = do.call(survival::coxph, list(formula = f, data = cdata_x))
    
    res.cox <- coxph(Surv(event, s) ~ ETP + Z_expres, data =  cdata_x,na.action=na.exclude)
    
   # print(Z_str)
    print(summary(res.cox))
    

    p_x = summary(res.cox)$coefficients[,5][[2]]
    p_y = summary(res.cox)$coefficients[,5][[1]]
    coef_x = summary(res.cox)$conf.int[,1][[2]]
    coef_ETP_x = summary(res.cox)$conf.int[,1][["ETPETP"]]

    lower_x  = summary(res.cox)$conf.int[,3][[2]]
    upper_x = summary(res.cox)$conf.int[,4][[2]]
    lower_ETP_x = summary(res.cox)$conf.int[,3][["ETPETP"]]
    upper_ETP_x = summary(res.cox)$conf.int[,4][["ETPETP"]]

    signif_x = 3
    return(c(signif(p_x, signif_x), signif(p_y, signif_x), 
             signif(coef_x, signif_x), signif(coef_ETP_x, signif_x),
             signif(lower_x, signif_x),signif(upper_x, signif_x),
             signif(lower_ETP_x, signif_x), signif(upper_ETP_x, signif_x)))
    
}


In [5]:
## -- OS
vec_1 = testCOX(scstrata_Z, include_near=FALSE, status_str="OS.status", event_str = "OS")
vec_2 = testCOX(scstrata_Z, include_near=TRUE, status_str="OS.status", event_str = "OS")

vec_3 = testCOX(scvec_Z, include_near=FALSE, status_str="OS.status", event_str = "OS")
vec_4 = testCOX(scvec_Z, include_near=TRUE, status_str="OS.status", event_str = "OS")


## -- EFS
vec_5 = testCOX(scstrata_Z, include_near=FALSE, status_str="EFS.status", event_str = "EFS")
vec_6 = testCOX(scstrata_Z, include_near=TRUE, status_str="EFS.status", event_str = "EFS")

vec_7 = testCOX(scvec_Z, include_near=FALSE, status_str="EFS.status", event_str = "EFS")
vec_8 = testCOX(scvec_Z, include_near=TRUE, status_str="EFS.status", event_str = "EFS")


cox_df = data.frame(Z_tert = vec_1,
           Z_tert_near = vec_2,
           Z_cont = vec_3,
           Z_cont_near = vec_4,
           Z_tert_efs = vec_5,
           Z_tert_efs_near = vec_6, 
           Z_cont_efs = vec_7, 
           Z_cont_efs_near = vec_8, 
                    
                    row.names = c("P-value", "P-value (ETP)", "Coef. ZBTB16", "Coef. ETP",
                                            "Lower .95 ZBTB16", "Higher .95 ZBTB16",
                                             "Lower .95 ETP", "Higher .95 ETP"))
cox_df <- t(cox_df)

Call:
coxph(formula = Surv(event, s) ~ ETP + Z_expres, data = cdata_x, 
    na.action = na.exclude)

  n= 581, number of events= 72 
   (307 observations deleted due to missingness)

                coef exp(coef) se(coef)     z Pr(>|z|)    
ETPETP        0.1513    1.1634   0.3008 0.503 0.614843    
Z_expres ^ Hi 1.0176    2.7667   0.2683 3.793 0.000149 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

              exp(coef) exp(-coef) lower .95 upper .95
ETPETP            1.163     0.8596    0.6452     2.098
Z_expres ^ Hi     2.767     0.3614    1.6353     4.681

Concordance= 0.629  (se = 0.029 )
Likelihood ratio test= 19.4  on 2 df,   p=6e-05
Wald test            = 17.91  on 2 df,   p=1e-04
Score (logrank) test = 19.67  on 2 df,   p=5e-05

Call:
coxph(formula = Surv(event, s) ~ ETP + Z_expres, data = cdata_x, 
    na.action = na.exclude)

  n= 697, number of events= 82 
   (342 observations deleted due to missingness)

                coef exp(coef) se(coef)   

In [6]:
column_ordering <- c("P-value", "Coef. ZBTB16", "Lower .95 ZBTB16", "Higher .95 ZBTB16","P-value (ETP)", "Coef. ETP", "Lower .95 ETP", "Higher .95 ETP")
cox_df <- as.data.frame(cox_df[,column_ordering])

colnames(cox_df) <- c("P-value", "Coef.", "Lower .95", "Upper .95", "P-value", "Coef.", "Lower .95", "Upper .95")

cox_df

,P-value,Coef.,Lower .95,Upper .95,P-value,Coef.,Lower .95,Upper .95
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Z_tert,1.49e-04,2.77,1.64,4.68,0.615,1.16,0.645,2.10
Z_tert_near,5.15e-04,2.41,1.47,3.95,0.265,1.38,0.782,2.44
Z_cont,3.09e-05,1.17,1.09,1.26,0.645,1.14,0.653,1.99
Z_cont_near,1.25e-04,1.14,1.07,1.22,0.250,1.37,0.803,2.33
Z_tert_efs,1.10e-03,2.05,1.33,3.17,0.389,1.26,0.748,2.11
Z_tert_efs_near,4.30e-04,2.02,1.36,2.98,0.341,1.27,0.777,2.08
Z_cont_efs,2.65e-04,1.13,1.06,1.20,0.568,1.15,0.707,1.88
Z_cont_efs_near,4.76e-06,1.14,1.08,1.21,0.566,1.15,0.720,1.82


### Save

In [7]:
write.csv(cox_df,paste0(save_dir,"SuppTable11_and_Fig4D.csv"), sep=",")

Warning message in write.csv(cox_df, paste0(save_dir, "SuppTable11_and_Fig4D.csv"), :
“attempt to set 'sep' ignored”


<br>

<br>

# Fig 4E & SuppTable13

### Set up

In [8]:
Z_modules <- getGmt(paste0("/lustre/scratch127/recovered/scratch126/lustre/scratch126/casm/team274sb/hw12/T_ALL/modules/", 
                     "ZBTB16_modules_refined.gmt"))


teachy_df = read.table("/lustre/scratch127/recovered/scratch126/lustre/scratch126/casm/team274sb/project_folders/T_ALL_paper/data/teachy_sig.csv", sep=",", header=TRUE)
teachy_sig = teachy_df$gene

teachy_df_17 = read.table("/lustre/scratch127/recovered/scratch126/lustre/scratch126/casm/team274sb/project_folders/T_ALL_paper/data/teachy_sig_17.csv", sep=",", header=TRUE)
teachy_17_sig = teachy_df_17$gene

teachy_9_sig = c("CD33", "IL3RA", "PTPRC", "HLA-DRA", "CD3E", "CD8A", "CD2","CD4","CD1A")



### --- Score
library(singscore)

cpm_data <- UPenn_dge$logCPM

## -- Rank genes
eranks = rankGenes(cpm_data, stableGenes=getStableGenes(10, type = 'blood'))
#eranks = rankGenes(cpm_data)

## -- Score genest
T_scores_vec = simpleScore(eranks, upSet=teachy_sig, centerScore=FALSE)$TotalScore
T17_scores_vec = simpleScore(eranks, upSet=teachy_17_sig, centerScore=FALSE)$TotalScore
T9_scores_vec = simpleScore(eranks, upSet=teachy_9_sig, centerScore=FALSE)$TotalScore
Z_scores_vec = simpleScore(eranks, upSet=geneIds(Z_modules[["refinedES_module_up"]]), centerScore=FALSE)$TotalScore

names(T_scores_vec) <- colnames(eranks)
names(T17_scores_vec) <- colnames(eranks)
names(T9_scores_vec) <- colnames(eranks)
names(Z_scores_vec) <- colnames(eranks)

Warning message in checkGenes(upSet, rownames(rankData)):
“4 genes missing: L1TD1, PTH2, MTRNR2L8, AC103591.3”
Warning message in checkGenes(upSet, rownames(rankData)):
“1 genes missing: IL3RA”
Warning message in checkGenes(upSet, rownames(rankData)):
“2 genes missing: TRBV2, NNMT”


### Split data

In [9]:
cutquantile = 0.3333
## -- By ZBTB16

scvec_Z = UPenn_dge$logCPM["ZBTB16",]
scstrata_Z = rep(NA, length(scvec_Z))
scstrata_Z[scvec_Z < quantile(scvec_Z, cutquantile)] = 'Low'
scstrata_Z[scvec_Z > quantile(scvec_Z, 1 - cutquantile)] = 'High'

## -- Rename
scstrata_Z = gsub('High', ' ^ Hi', scstrata_Z)
scstrata_Z = gsub('Low', ' ^ Lo', scstrata_Z)
names(scstrata_Z) <- names(scvec_Z)

UPenn_dge$samples$survival_group <- as.vector(scstrata_Z[UPenn_dge$samples$sample_id])
UPenn_dge$samples$ZBTB16_exprs <-  as.vector(UPenn_dge$logCPM["ZBTB16",])

## -- By our module score
scvec_Z = Z_scores_vec
scstrata_Z = rep("Mid", length(scvec_Z))
scstrata_Z[scvec_Z < quantile(scvec_Z, cutquantile)] = 'Low'
scstrata_Z[scvec_Z > quantile(scvec_Z, 1 - cutquantile)] = 'High'
names(scstrata_Z) <- rownames(UPenn_dge$samples)
UPenn_dge$samples$Z_module_group <- as.vector(scstrata_Z[UPenn_dge$samples$sample_id])

## -- By their module score

scstrata_T = rep("Mid", length(T_scores_vec))
scstrata_T[T_scores_vec < quantile(T_scores_vec, cutquantile)] = 'Low'
scstrata_T[T_scores_vec > quantile(T_scores_vec, 1 - cutquantile)] = 'High'
names(scstrata_T) <- rownames(UPenn_dge$samples)
UPenn_dge$samples$T_module_group <- as.vector(scstrata_T[UPenn_dge$samples$sample_id])

## -- By their module score -- BMP-17

scstrata_T17 = rep("Mid", length(T17_scores_vec))
scstrata_T17[T17_scores_vec < quantile(T17_scores_vec, cutquantile)] = 'Low'
scstrata_T17[T17_scores_vec > quantile(T17_scores_vec, 1 - cutquantile)] = 'High'
names(scstrata_T17) <- rownames(UPenn_dge$samples)
UPenn_dge$samples$T_17_module_group <- as.vector(scstrata_T17[UPenn_dge$samples$sample_id])

## -- By their module score -- BMP-surface-9
scstrata_T9 = rep("Mid", length(T9_scores_vec))
scstrata_T9[T9_scores_vec < quantile(T9_scores_vec, cutquantile)] = 'Low'
scstrata_T9[T9_scores_vec > quantile(T9_scores_vec, 1 - cutquantile)] = 'High'
names(scstrata_T9) <- rownames(UPenn_dge$samples)
UPenn_dge$samples$T_9_module_group <- as.vector(scstrata_T9[UPenn_dge$samples$sample_id])


## -- By ETP  // exclude Near-ETP & Unknown

scstrata_ETP = rep(NA, nrow(UPenn_dge$samples))
scstrata_ETP[UPenn_dge$samples$ETP_status=="ETP"] = 'ETP'
scstrata_ETP[UPenn_dge$samples$ETP_status=="Non-ETP"] = 'Non-ETP'
names(scstrata_ETP) <- rownames(UPenn_dge$samples)

## -- By ETP  // exclude unknown but include Near-ETP!
scstrata_ETP_near = rep(NA, nrow(UPenn_dge$samples))
scstrata_ETP_near[UPenn_dge$samples$ETP_status=="ETP"] = 'ETP'
scstrata_ETP_near[UPenn_dge$samples$ETP_status %in% c("Non-ETP", "Near-ETP")] = 'Non-ETP'
names(scstrata_ETP_near) <- rownames(UPenn_dge$samples)


### Run cox

In [10]:
#### ---- FUNCTIONS
do_cox <- function(x_str, y_str, include_ETP=FALSE, status_str="OS.status", event_str = "OS"){
    
    # -- Create into survival data structure
    cdata_x$s = grepl("1", cdata_x[[status_str]], ignore.case = TRUE)
    cdata_x$event <- as.vector(cdata_x[[event_str]])
    cdata_x = cdata_x[, c("event", "s", "Z_cont", "Z_expres","T_cont", "T17_cont", "T_cont_s", "T17_cont_s", "Z_cont_s", "T9_cont_s", "T9_cont", "ETP")]
    
    cdata_x$x <- cdata_x[[x_str]]
    cdata_x$y <- cdata_x[[y_str]]
    
    ## -- Run model
    if (include_ETP){
        cdata_x$ETP <- relevel(cdata_x$ETP, "Non-ETP")
        res.cox <- coxph(Surv(event, s) ~ x + y + ETP, data =  cdata_x,na.action=na.exclude)
    } else {
        res.cox <- coxph(Surv(event, s) ~ x + y, data =  cdata_x,na.action=na.exclude)
    }
    print(paste0("X = ",x_str))
    print(paste0("Y = ",y_str))
    print(summary(res.cox)) 
    
    return(res.cox)

}

get_vec <- function(model_x, str_x){
     signif(summary(model_x)$coefficients[,5][[str_x]], 3)
    
    return(c(c(signif(summary(model_x)$conf.int[,1][[str_x]], 3), 
               signif(summary(model_x)$conf.int[,3][[str_x]], 3),  
               signif(summary(model_x)$conf.int[,4][[str_x]], 3),
              signif(summary(model_x)$coefficients[,5][[str_x]], 3))))
}


### Prep

In [11]:
## Not factors because continuous
UPenn_dge$samples$T_cont <- as.vector(T_scores_vec[rownames(UPenn_dge$sample)])
UPenn_dge$samples$T17_cont <- as.vector(T17_scores_vec[rownames(UPenn_dge$sample)])
UPenn_dge$samples$T9_cont <- as.vector(T9_scores_vec[rownames(UPenn_dge$sample)])
UPenn_dge$samples$Z_cont <- as.vector(Z_scores_vec[rownames(UPenn_dge$sample)])

UPenn_dge$samples$T_cont_s <- scale(as.vector(T_scores_vec[rownames(UPenn_dge$sample)]))
UPenn_dge$samples$T17_cont_s <- scale(as.vector(T17_scores_vec[rownames(UPenn_dge$sample)]))
UPenn_dge$samples$T9_cont_s <- scale(as.vector(T9_scores_vec[rownames(UPenn_dge$sample)]))
UPenn_dge$samples$Z_cont_s <- scale(as.vector(Z_scores_vec[rownames(UPenn_dge$sample)]))

UPenn_dge$samples$Z_expres <- as.vector(UPenn_dge$logCPM["ZBTB16",][rownames(UPenn_dge$sample)])
UPenn_dge$samples$ETP <- factor(scstrata_ETP_near[rownames(UPenn_dge$sample)])

cdata_x = UPenn_dge$samples

#### OS

In [12]:
## -- OS

model_1 <- do_cox("Z_cont_s", "T_cont_s",include_ETP=FALSE)
#model_2 <- do_cox("Z_expres", "T_cont_s",include_ETP=FALSE)

model_2 <- do_cox("Z_cont_s", "T17_cont_s",include_ETP=FALSE)
#model_4 <- do_cox("Z_expres", "T17_cont_s",include_ETP=FALSE)

model_3 <- do_cox("Z_cont_s", "T9_cont_s",include_ETP=FALSE)

[1] "X = Z_cont_s"
[1] "Y = T_cont_s"
Call:
coxph(formula = Surv(event, s) ~ x + y, data = cdata_x, na.action = na.exclude)

  n= 1175, number of events= 127 

     coef exp(coef) se(coef)     z Pr(>|z|)    
x 0.36953   1.44706  0.08956 4.126 3.69e-05 ***
y 0.11519   1.12209  0.10059 1.145    0.252    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

  exp(coef) exp(-coef) lower .95 upper .95
x     1.447     0.6911    1.2141     1.725
y     1.122     0.8912    0.9213     1.367

Concordance= 0.635  (se = 0.024 )
Likelihood ratio test= 29.84  on 2 df,   p=3e-07
Wald test            = 32.91  on 2 df,   p=7e-08
Score (logrank) test = 34.36  on 2 df,   p=3e-08

[1] "X = Z_cont_s"
[1] "Y = T17_cont_s"
Call:
coxph(formula = Surv(event, s) ~ x + y, data = cdata_x, na.action = na.exclude)

  n= 1175, number of events= 127 

    coef exp(coef) se(coef)     z Pr(>|z|)   
x 0.3074    1.3599   0.1120 2.746  0.00604 **
y 0.1680    1.1830   0.1171 1.435  0.15115   
---
Signif. code

In [13]:
### -- make dataframe

cox_df = data.frame(m1_Z = get_vec(model_1, "x"),
                    m1_T = get_vec(model_1, "y"),
                #    m1_ETP = get_vec(model_1, "ETPETP"),
                    
                #    m2_Zexpres = get_vec(model_2, "x"),
                #    m2_T = get_vec(model_2, "y"),
                #    m2_ETP = get_vec(model_2, "ETPETP"),
                    
                    m3_Z = get_vec(model_2, "x"),
                    m3_T17 = get_vec(model_2, "y"),
                #    m3_ETP = get_vec(model_3, "ETPETP"),
                    
               #     m4_Zexpres = get_vec(model_4, "x"),
               #     m4_T17 = get_vec(model_4, "y"),
                #    m4_ETP = get_vec(model_4, "ETPETP"),
                    
                    m5_Z = get_vec(model_3, "x"),
                    m5_T9 = get_vec(model_3, "y"),
                #    m5_ETP = get_vec(model_5, "ETPETP"),
                    
               #     m6_Zexpres = get_vec(model_6, "x"),
               #     m6_T9 = get_vec(model_6, "y"),
                  #  m6_ETP = get_vec(model_6, "ETPETP"),
                    
                    
                    row.names = c("Coef.", "Lower .95", "Higher .95", "P-value"))
cox_df <- as.data.frame(t(cox_df))

cox_df$Model <- unlist(lapply(rownames(cox_df), function(x){strsplit(x,"_")[[1]][1]}))

var_dict <- setNames(c("Module", "ZBTB16", "BMP-119", "BMP-17", "BMP-surface-9","ETP"), c("Z", "Zexpres", "T", "T17", "T9", "ETP"))
cox_df$Variable <- unlist(lapply(rownames(cox_df), function(x){var_dict[[strsplit(x,"_")[[1]][2]]]}))

column_ordering <- c("Model", "Variable", "Coef.", "Lower .95", "Higher .95", "P-value")
cox_df <- cox_df[,column_ordering]

cox_df

,Model,Variable,Coef.,Lower .95,Higher .95,P-value
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
m1_Z,m1,Module,1.450,1.210,1.72,3.69e-05
m1_T,m1,BMP-119,1.120,0.921,1.37,2.52e-01
m3_Z,m3,Module,1.360,1.090,1.69,6.04e-03
m3_T17,m3,BMP-17,1.180,0.940,1.49,1.51e-01
m5_Z,m5,Module,1.520,1.310,1.77,4.37e-08
m5_T9,m5,BMP-surface-9,0.984,0.820,1.18,8.61e-01


#### Save

In [14]:
write.csv(cox_df,paste0(save_dir,"SuppTable13_OS.csv"), sep=",")

Warning message in write.csv(cox_df, paste0(save_dir, "SuppTable13_OS.csv"), sep = ","):
“attempt to set 'sep' ignored”


<br>

#### EFS

In [15]:
model_4 <- do_cox("Z_cont_s", "T_cont_s",include_ETP=FALSE, status_str="EFS.status", event_str = "EFS")
#model_2 <- do_cox("Z_expres", "T_cont_s",include_ETP=FALSE, status_str="EFS.status", event_str = "EFS")

model_5 <- do_cox("Z_cont_s", "T17_cont_s",include_ETP=FALSE, status_str="EFS.status", event_str = "EFS")
#model_4 <- do_cox("Z_expres", "T17_cont_s",include_ETP=FALSE, status_str="EFS.status", event_str = "EFS")

model_6 <- do_cox("Z_cont_s", "T9_cont_s",include_ETP=FALSE, status_str="EFS.status", event_str = "EFS")

[1] "X = Z_cont_s"
[1] "Y = T_cont_s"
Call:
coxph(formula = Surv(event, s) ~ x + y, data = cdata_x, na.action = na.exclude)

  n= 1175, number of events= 194 

     coef exp(coef) se(coef)     z Pr(>|z|)    
x 0.40127   1.49372  0.07400 5.423 5.86e-08 ***
y 0.09457   1.09919  0.08219 1.151     0.25    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

  exp(coef) exp(-coef) lower .95 upper .95
x     1.494     0.6695    1.2921     1.727
y     1.099     0.9098    0.9357     1.291

Concordance= 0.635  (se = 0.021 )
Likelihood ratio test= 47.77  on 2 df,   p=4e-11
Wald test            = 53.5  on 2 df,   p=2e-12
Score (logrank) test = 55.76  on 2 df,   p=8e-13

[1] "X = Z_cont_s"
[1] "Y = T17_cont_s"
Call:
coxph(formula = Surv(event, s) ~ x + y, data = cdata_x, na.action = na.exclude)

  n= 1175, number of events= 194 

     coef exp(coef) se(coef)     z Pr(>|z|)    
x 0.36998   1.44771  0.09206 4.019 5.85e-05 ***
y 0.11091   1.11730  0.09637 1.151     0.25    
---
Signif.

In [16]:
### -- make dataframe

cox_df = data.frame(m1_Z = get_vec(model_4, "x"),
                    m1_T = get_vec(model_4, "y"),
                #    m1_ETP = get_vec(model_1, "ETPETP"),
                    
              #      m2_Zexpres = get_vec(model_2, "x"),
               #     m2_T = get_vec(model_2, "y"),
                #    m2_ETP = get_vec(model_2, "ETPETP"),
                    
                    m3_Z = get_vec(model_5, "x"),
                    m3_T17 = get_vec(model_5, "y"),
                #    m3_ETP = get_vec(model_3, "ETPETP"),
                    
               #     m4_Zexpres = get_vec(model_4, "x"),
               #     m4_T17 = get_vec(model_4, "y"),
                #    m4_ETP = get_vec(model_4, "ETPETP"),
                    
                    m5_Z = get_vec(model_6, "x"),
                    m5_T9 = get_vec(model_6, "y"),
                #    m5_ETP = get_vec(model_5, "ETPETP"),
                    
                #    m6_Zexpres = get_vec(model_6, "x"),
                #    m6_T9 = get_vec(model_6, "y"),
                  #  m6_ETP = get_vec(model_6, "ETPETP"),
                    
                    
                    row.names = c("Coef.", "Lower .95", "Higher .95", "P-value"))
cox_df <- as.data.frame(t(cox_df))

cox_df$Model <- unlist(lapply(rownames(cox_df), function(x){strsplit(x,"_")[[1]][1]}))

var_dict <- setNames(c("Module", "ZBTB16", "BMP-119", "BMP-17", "BMP-surface-9","ETP"), c("Z", "Zexpres", "T", "T17", "T9", "ETP"))
cox_df$Variable <- unlist(lapply(rownames(cox_df), function(x){var_dict[[strsplit(x,"_")[[1]][2]]]}))

column_ordering <- c("Model", "Variable", "Coef.", "Lower .95", "Higher .95", "P-value")
cox_df <- cox_df[,column_ordering]

cox_df

,Model,Variable,Coef.,Lower .95,Higher .95,P-value
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
m1_Z,m1,Module,1.490,1.290,1.73,5.86e-08
m1_T,m1,BMP-119,1.100,0.936,1.29,2.50e-01
m3_Z,m3,Module,1.450,1.210,1.73,5.85e-05
m3_T17,m3,BMP-17,1.120,0.925,1.35,2.50e-01
m5_Z,m5,Module,1.560,1.380,1.77,1.32e-12
m5_T9,m5,BMP-surface-9,0.988,0.853,1.14,8.70e-01


#### Save

In [17]:
write.csv(cox_df,paste0(save_dir,"SuppTable13_EFS.csv"), sep=",")

Warning message in write.csv(cox_df, paste0(save_dir, "SuppTable13_EFS.csv"), sep = ","):
“attempt to set 'sep' ignored”


<br>

<br>

# SuppFig 6A & SuppTable12

### Get transcriptional subtype

In [18]:
UPenn_dge$samples$Subtype <- as.vector(setNames(bulk_metadat_1$Reviewed.subtype,
                                bulk_metadat_1$sample)[UPenn_dge$samples$sample_id])
UPenn_dge$samples$transcriptional_ETP <- ifelse(UPenn_dge$samples$Subtype=="ETP-like","ETP-like", "Other")

table(UPenn_dge$samples$transcriptional_ETP)


ETP-like    Other 
     200      975 

### Split data

In [19]:
cutquantile = 0.3333

## -- By ZBTB16

scvec_Z = UPenn_dge$logCPM["ZBTB16",]
scstrata_Z = rep(NA, length(scvec_Z))
scstrata_Z[scvec_Z < quantile(scvec_Z, cutquantile)] = 'Low'
scstrata_Z[scvec_Z > quantile(scvec_Z, 1 - cutquantile)] = 'High'

## -- Rename
scstrata_Z = gsub('High', ' ^ Hi', scstrata_Z)
scstrata_Z = gsub('Low', ' ^ Lo', scstrata_Z)
names(scstrata_Z) <- names(scvec_Z)

UPenn_dge$samples$survival_group <- as.vector(scstrata_Z[UPenn_dge$samples$sample_id])
UPenn_dge$samples$ZBTB16_exprs <-  as.vector(UPenn_dge$logCPM["ZBTB16",])

## -- By ETP  // exclude Near-ETP & Unknown

scstrata_ETP = rep(NA, nrow(UPenn_dge$samples))
scstrata_ETP[UPenn_dge$samples$transcriptional_ETP=="ETP-like"] = 'ETP-like'
scstrata_ETP[UPenn_dge$samples$transcriptional_ETP=="Other"] = 'Other'
names(scstrata_ETP) <- rownames(UPenn_dge$samples)


### Run cox

In [20]:
testCOX <- function(ZBTB16_vec_x, ETP_vec_x, status_str="OS.status", event_str = "OS"){
    ### Alternative:
    # - status_str = "EFS.status"
    # - event_str = "EFS"
    
    
    ## -- Prep
    cdata_x = UPenn_dge$samples[rownames(UPenn_dge$samples) %in% names(ETP_vec_x[!(is.na(ETP_vec_x))]),]



    ## -- Check values
    if (length(cdata_x[which((cdata_x[, event_str] < 0) == TRUE), 
                   "short_name"]) > 0 ){
            print("Some patients have negative values for event")
    }
    
    # -- Create into survival data structure
    cdata_x$s = grepl("1", cdata_x[[status_str]], ignore.case = TRUE)
    cdata_x$ETP = factor(as.vector(ETP_vec_x[rownames(cdata_x)]), levels=unique(as.vector(ETP_vec_x[rownames(cdata_x)])))

    if (is.character(ZBTB16_vec_x[[1]])){
        cdata_x$Z_expres <- factor(as.vector(ZBTB16_vec_x[rownames(cdata_x)]))
        cdata_x$Z_expres <- relevel(cdata_x$Z_expres, " ^ Lo")
    } else {
        cdata_x$Z_expres = as.vector(ZBTB16_vec_x[rownames(cdata_x)])
    }

    cdata_x$event <- as.vector(cdata_x[[event_str]])
    cdata_x = cdata_x[, c("event", "s", "ETP","Z_expres")]
    cdata_x = cdata_x[order(cdata_x$ETP),]

    #if (class(cdata_x$Z_expres)=="numeric"){
    #    Z_str = "Z_expres"
    #} else {
    #    cdata_x$Z_expres <- as.factor(cdata_x$Z_expres)
    #    cdata_x = cdata_x[rev(order(cdata_x$Z_expres)),]
    #    Z_str = paste0("Z_expres",unique(cdata_x$Z_expres[!(is.na(cdata_x$Z_expres))])[2])    
    #}
    
    ## -- Set formula
    f = as.formula("Surv(event, s) ~ ETP + Z_expres")
    
    sfit = do.call(survival::coxph, list(formula = f, data = cdata_x))
    
    res.cox <- coxph(Surv(event, s) ~ ETP + Z_expres, data =  cdata_x,na.action=na.exclude)
    
   # print(Z_str)
    print(summary(res.cox))
    

    p_x = summary(res.cox)$coefficients[,5][[2]]
    p_y = summary(res.cox)$coefficients[,5][[1]]
    coef_x = summary(res.cox)$conf.int[,1][[2]]
    coef_ETP_x = summary(res.cox)$conf.int[,1][["ETPETP-like"]]

    lower_x  = summary(res.cox)$conf.int[,3][[2]]
    upper_x = summary(res.cox)$conf.int[,4][[2]]
    lower_ETP_x = summary(res.cox)$conf.int[,3][["ETPETP-like"]]
    upper_ETP_x = summary(res.cox)$conf.int[,4][["ETPETP-like"]]

    signif_x = 3
    return(c(signif(p_x, signif_x), signif(p_y, signif_x), 
             signif(coef_x, signif_x), signif(coef_ETP_x, signif_x),
             signif(lower_x, signif_x),signif(upper_x, signif_x),
             signif(lower_ETP_x, signif_x), signif(upper_ETP_x, signif_x)))
    
}

In [21]:
vec_1 = testCOX(scvec_Z, scstrata_ETP, status_str="OS.status", event_str = "OS")
#vec_2 = testCOX(scvec_Z, scstrata_ETP, status_str="OS.status", event_str = "OS")
vec_3 = testCOX(scstrata_Z, scstrata_ETP, status_str="OS.status", event_str = "OS")
#vec_4 = testCOX(scstrata_Z, scstrata_ETP, status_str="OS.status", event_str = "OS")

vec_5 = testCOX(scvec_Z, scstrata_ETP, status_str="EFS.status", event_str = "EFS")
#vec_6 = testCOX(scvec_Z, scstrata_ETP, status_str="EFS.status", event_str = "EFS")
vec_7 = testCOX(scstrata_Z, scstrata_ETP, status_str="EFS.status", event_str = "EFS")
#vec_8 = testCOX(scstrata_Z, scstrata_ETP, status_str="EFS.status", event_str = "EFS")

Call:
coxph(formula = Surv(event, s) ~ ETP + Z_expres, data = cdata_x, 
    na.action = na.exclude)

  n= 1175, number of events= 127 

               coef exp(coef) se(coef)     z Pr(>|z|)   
ETPETP-like 0.22412   1.25123  0.24213 0.926  0.35464   
Z_expres    0.10729   1.11325  0.03711 2.891  0.00384 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

            exp(coef) exp(-coef) lower .95 upper .95
ETPETP-like     1.251     0.7992    0.7784     2.011
Z_expres        1.113     0.8983    1.0352     1.197

Concordance= 0.604  (se = 0.025 )
Likelihood ratio test= 16.9  on 2 df,   p=2e-04
Wald test            = 17.83  on 2 df,   p=1e-04
Score (logrank) test = 18.37  on 2 df,   p=1e-04

Call:
coxph(formula = Surv(event, s) ~ ETP + Z_expres, data = cdata_x, 
    na.action = na.exclude)

  n= 784, number of events= 92 
   (391 observations deleted due to missingness)

                coef exp(coef) se(coef)     z Pr(>|z|)   
ETPETP-like   0.2690    1.3087   0.2438 1.1

### Save

In [22]:
cox_df = data.frame(a_Z_cont_OS = vec_1,
          # b_Z_cont_OS_near = vec_2,
           b_Z_tert_OS = vec_3,
         # d_Z_tert_near = vec_4,
           c_Z_cont_EFS = vec_5,
          # f_Z_EFS_cont_near = vec_6,
           d_Z_tert_EFS = vec_7,
         #  h_Z_tert_EFS_near = vec_8,
                    row.names = c("P-value", "P-value (ETP)", "Coef. ZBTB16", "Coef. ETP",
                                            "Lower .95 ZBTB16", "Higher .95 ZBTB16",
                                             "Lower .95 ETP", "Higher .95 ETP"))
cox_df <- t(cox_df)

column_ordering <- c("P-value", "Coef. ZBTB16", "Lower .95 ZBTB16", "Higher .95 ZBTB16","P-value (ETP)", "Coef. ETP", "Lower .95 ETP", "Higher .95 ETP")

cox_df[,column_ordering]

,P-value,Coef. ZBTB16,Lower .95 ZBTB16,Higher .95 ZBTB16,P-value (ETP),Coef. ETP,Lower .95 ETP,Higher .95 ETP
a_Z_cont_OS,0.00384,1.11,1.04,1.20,0.35500,1.25,0.778,2.01
b_Z_tert_OS,0.00374,2.06,1.26,3.37,0.27000,1.31,0.811,2.11
c_Z_cont_EFS,0.00275,1.10,1.03,1.16,0.01320,1.61,1.110,2.35
d_Z_tert_EFS,0.02340,1.59,1.06,2.38,0.00175,1.88,1.270,2.78


In [23]:
write.csv(cox_df,paste0(save_dir,"SuppTable12_and_SuppFig6A.csv"), sep=",")

Warning message in write.csv(cox_df, paste0(save_dir, "SuppTable12_and_SuppFig6A.csv"), :
“attempt to set 'sep' ignored”
